In [ ]:
!pip install tensorboard_logger

In [1]:
import argparse
import torch
import torch.nn as nn
import tensorboard_logger
from nets import UnitedNet
from torch.utils.data import dataloader
from dataset import EGFRDataset, train_validation_split
import torch.optim as optim
from metrics import *
import utils
import matplotlib.pyplot as plt
import os
import warnings
from sklearn.metrics import precision_recall_curve
plt.switch_backend('agg')
warnings.filterwarnings('ignore')

In [2]:
class ArgStruct():
    def __init__(self):
        self.dataset = 'data/egfr_10_full_ft_pd_lines.json'
        self.epochs = 500
        self.batchsize = 128
        self.opt = 'adam'
        self.gpu = 'store_true'
        self.hashcode = 'TEST'
        self.lr = 1e-5
        self.mode = 'train'
        self.model_path = ''
        self.use_mat = True
        self.use_mord = True

args = ArgStruct()

***CROSS_VALIDATION (DEFAULT)***

In [3]:
from cross_val import *

if torch.cuda.is_available():
    train_device = 'cuda'
    val_device = 'cuda'
else:
    train_device = 'cpu'
    val_device = 'cpu'

tensorboard_logger.configure('logs/' + args.hashcode)

metrics_cv_dict = {'sensitivity': sensitivity_cv,
                   'specificity': specificity_cv,
                   'accuracy': accuracy_cv,
                   'mcc': mcc_cv,
                   'auc': auc_cv}

train_cv_metrics = []
val_cv_metrics = []
best_cv = []
fold = 0
for train_data, val_data in train_cross_validation_split(args.dataset):
    fold += 1
    train_dataset = EGFRDataset(train_data)
    val_dataset = EGFRDataset(val_data)
    train_metrics, val_metrics = train_validate_united(train_dataset,
                                                       val_dataset,
                                                       train_device,
                                                       val_device,
                                                       args.use_mat,
                                                       args.use_mord,
                                                       args.opt,
                                                       int(args.epochs),
                                                       int(args.batchsize),
                                                       {'sensitivity': sensitivity, 'specificity': specificity,
                                                        'accuracy': accuracy, 'mcc': mcc, 'auc': auc},
                                                       args.hashcode,
                                                       args.lr,
                                                       fold)
    train_cv_metrics.append(train_metrics)
    val_cv_metrics.append(val_metrics)

    # VALIDATE in BEST MODEL
    filename = "data/trained_models/model_" + args.hashcode + '_' + str(fold) + "_BEST"
    y_pred = predict(val_dataset, filename, use_mat=args.use_mat, use_mord=args.use_mord)
    y_true = val_dataset.label
    bestcv = {}
    for metric, metric_fun in metrics_cv_dict.items():
        bestcv[metric] = metric_fun(y_true, y_pred)

    best_cv.append(bestcv)

train_metrics_df = pd.DataFrame(train_cv_metrics)
val_metrics_df = pd.DataFrame(val_cv_metrics)
best_cv_df = pd.DataFrame(best_cv)
cv_metrics_df = pd.DataFrame([train_metrics_df.mean(),
                              val_metrics_df.mean(),
                              best_cv_df.mean()],
                             index=['train', 'val', 'load'])
print(cv_metrics_df[['sensitivity', 'specificity', 'accuracy', 'mcc', 'auc']])

entered mord branch
entered mat branch
FOLD 1 -- EPOCH 0 -- TRAINING
FOLD 1 EPOCH 0 -- VALIDATING
{"metric": "train_loss", "value": 0.458124, "epoch": 1}
{"metric": "val_loss", "value": 0.455397, "epoch": 1}
{"metric": "train_sensitivity", "value": 0.559406, "epoch": 1}
{"metric": "val_sensitivity", "value": 0.794118, "epoch": 1}
{"metric": "train_specificity", "value": 0.408291, "epoch": 1}
{"metric": "val_specificity", "value": 0.214047, "epoch": 1}
{"metric": "train_accuracy", "value": 0.430158, "epoch": 1}
{"metric": "val_accuracy", "value": 0.298571, "epoch": 1}
{"metric": "train_mcc", "value": -0.023080, "epoch": 1}
{"metric": "val_mcc", "value": 0.007037, "epoch": 1}
{"metric": "train_auc", "value": 0.473538, "epoch": 1}
{"metric": "val_auc", "value": 0.484130, "epoch": 1}
Save done!
FOLD 1 -- EPOCH 1 -- TRAINING
FOLD 1 EPOCH 1 -- VALIDATING
{"metric": "train_loss", "value": 0.433608, "epoch": 2}
{"metric": "val_loss", "value": 0.427321, "epoch": 2}
{"metric": "train_sensitivity

Save done!
FOLD 1 -- EPOCH 11 -- TRAINING
FOLD 1 EPOCH 11 -- VALIDATING
{"metric": "train_loss", "value": 0.389012, "epoch": 12}
{"metric": "val_loss", "value": 0.381058, "epoch": 12}
{"metric": "train_sensitivity", "value": 0.316832, "epoch": 12}
{"metric": "val_sensitivity", "value": 0.411765, "epoch": 12}
{"metric": "train_specificity", "value": 0.865578, "epoch": 12}
{"metric": "val_specificity", "value": 0.956522, "epoch": 12}
{"metric": "train_accuracy", "value": 0.786175, "epoch": 12}
{"metric": "val_accuracy", "value": 0.877143, "epoch": 12}
{"metric": "train_mcc", "value": 0.174681, "epoch": 12}
{"metric": "val_mcc", "value": 0.438756, "epoch": 12}
{"metric": "train_auc", "value": 0.679880, "epoch": 12}
{"metric": "val_auc", "value": 0.777526, "epoch": 12}
Save done!
FOLD 1 -- EPOCH 12 -- TRAINING
FOLD 1 EPOCH 12 -- VALIDATING
{"metric": "train_loss", "value": 0.387768, "epoch": 13}
{"metric": "val_loss", "value": 0.378470, "epoch": 13}
{"metric": "train_sensitivity", "value":

FOLD 1 EPOCH 22 -- VALIDATING
{"metric": "train_loss", "value": 0.363427, "epoch": 23}
{"metric": "val_loss", "value": 0.353604, "epoch": 23}
{"metric": "train_sensitivity", "value": 0.497525, "epoch": 23}
{"metric": "val_sensitivity", "value": 0.627451, "epoch": 23}
{"metric": "train_specificity", "value": 0.863484, "epoch": 23}
{"metric": "val_specificity", "value": 0.896321, "epoch": 23}
{"metric": "train_accuracy", "value": 0.810530, "epoch": 23}
{"metric": "val_accuracy", "value": 0.857143, "epoch": 23}
{"metric": "train_mcc", "value": 0.324553, "epoch": 23}
{"metric": "val_mcc", "value": 0.481007, "epoch": 23}
{"metric": "train_auc", "value": 0.777687, "epoch": 23}
{"metric": "val_auc", "value": 0.842777, "epoch": 23}
Save done!
FOLD 1 -- EPOCH 23 -- TRAINING
FOLD 1 EPOCH 23 -- VALIDATING
{"metric": "train_loss", "value": 0.360954, "epoch": 24}
{"metric": "val_loss", "value": 0.351813, "epoch": 24}
{"metric": "train_sensitivity", "value": 0.492574, "epoch": 24}
{"metric": "val_se

FOLD 1 EPOCH 33 -- VALIDATING
{"metric": "train_loss", "value": 0.335700, "epoch": 34}
{"metric": "val_loss", "value": 0.332274, "epoch": 34}
{"metric": "train_sensitivity", "value": 0.638614, "epoch": 34}
{"metric": "val_sensitivity", "value": 0.686275, "epoch": 34}
{"metric": "train_specificity", "value": 0.863065, "epoch": 34}
{"metric": "val_specificity", "value": 0.854515, "epoch": 34}
{"metric": "train_accuracy", "value": 0.830587, "epoch": 34}
{"metric": "val_accuracy", "value": 0.830000, "epoch": 34}
{"metric": "train_mcc", "value": 0.433665, "epoch": 34}
{"metric": "val_mcc", "value": 0.457435, "epoch": 34}
{"metric": "train_auc", "value": 0.842301, "epoch": 34}
{"metric": "val_auc", "value": 0.862696, "epoch": 34}
Save done!
FOLD 1 -- EPOCH 34 -- TRAINING
FOLD 1 EPOCH 34 -- VALIDATING
{"metric": "train_loss", "value": 0.334424, "epoch": 35}
{"metric": "val_loss", "value": 0.330927, "epoch": 35}
{"metric": "train_sensitivity", "value": 0.643564, "epoch": 35}
{"metric": "val_se

FOLD 1 EPOCH 44 -- VALIDATING
{"metric": "train_loss", "value": 0.317542, "epoch": 45}
{"metric": "val_loss", "value": 0.313812, "epoch": 45}
{"metric": "train_sensitivity", "value": 0.720297, "epoch": 45}
{"metric": "val_sensitivity", "value": 0.725490, "epoch": 45}
{"metric": "train_specificity", "value": 0.853853, "epoch": 45}
{"metric": "val_specificity", "value": 0.841137, "epoch": 45}
{"metric": "train_accuracy", "value": 0.834527, "epoch": 45}
{"metric": "val_accuracy", "value": 0.824286, "epoch": 45}
{"metric": "train_mcc", "value": 0.480531, "epoch": 45}
{"metric": "val_mcc", "value": 0.467151, "epoch": 45}
{"metric": "train_auc", "value": 0.863720, "epoch": 45}
{"metric": "val_auc", "value": 0.873402, "epoch": 45}
Save done!
FOLD 1 -- EPOCH 45 -- TRAINING
FOLD 1 EPOCH 45 -- VALIDATING
{"metric": "train_loss", "value": 0.313478, "epoch": 46}
{"metric": "val_loss", "value": 0.312595, "epoch": 46}
{"metric": "train_sensitivity", "value": 0.715347, "epoch": 46}
{"metric": "val_se

...

FOLD 5 EPOCH 365 -- VALIDATING
{"metric": "train_loss", "value": 0.091765, "epoch": 366}
{"metric": "val_loss", "value": 0.209004, "epoch": 366}
{"metric": "train_sensitivity", "value": 0.953086, "epoch": 366}
{"metric": "val_sensitivity", "value": 0.772277, "epoch": 366}
{"metric": "train_specificity", "value": 0.969443, "epoch": 366}
{"metric": "val_specificity", "value": 0.907873, "epoch": 366}
{"metric": "train_accuracy", "value": 0.967072, "epoch": 366}
{"metric": "val_accuracy", "value": 0.888252, "epoch": 366}
{"metric": "train_mcc", "value": 0.876529, "epoch": 366}
{"metric": "val_mcc", "value": 0.609260, "epoch": 366}
{"metric": "train_auc", "value": 0.994497, "epoch": 366}
{"metric": "val_auc", "value": 0.930096, "epoch": 366}
FOLD 5 -- EPOCH 366 -- TRAINING
FOLD 5 EPOCH 366 -- VALIDATING
{"metric": "train_loss", "value": 0.090122, "epoch": 367}
{"metric": "val_loss", "value": 0.208789, "epoch": 367}
{"metric": "train_sensitivity", "value": 0.958025, "epoch": 367}
{"metric": 

FOLD 5 EPOCH 376 -- VALIDATING
{"metric": "train_loss", "value": 0.085503, "epoch": 377}
{"metric": "val_loss", "value": 0.209564, "epoch": 377}
{"metric": "train_sensitivity", "value": 0.960494, "epoch": 377}
{"metric": "val_sensitivity", "value": 0.762376, "epoch": 377}
{"metric": "train_specificity", "value": 0.972792, "epoch": 377}
{"metric": "val_specificity", "value": 0.911223, "epoch": 377}
{"metric": "train_accuracy", "value": 0.971009, "epoch": 377}
{"metric": "val_accuracy", "value": 0.889685, "epoch": 377}
{"metric": "train_mcc", "value": 0.890665, "epoch": 377}
{"metric": "val_mcc", "value": 0.608700, "epoch": 377}
{"metric": "train_auc", "value": 0.995809, "epoch": 377}
{"metric": "val_auc", "value": 0.929748, "epoch": 377}
FOLD 5 -- EPOCH 377 -- TRAINING
FOLD 5 EPOCH 377 -- VALIDATING
{"metric": "train_loss", "value": 0.083470, "epoch": 378}
{"metric": "val_loss", "value": 0.209409, "epoch": 378}
{"metric": "train_sensitivity", "value": 0.977778, "epoch": 378}
{"metric": 

FOLD 5 EPOCH 387 -- VALIDATING
{"metric": "train_loss", "value": 0.085615, "epoch": 388}
{"metric": "val_loss", "value": 0.211941, "epoch": 388}
{"metric": "train_sensitivity", "value": 0.970370, "epoch": 388}
{"metric": "val_sensitivity", "value": 0.762376, "epoch": 388}
{"metric": "train_specificity", "value": 0.971536, "epoch": 388}
{"metric": "val_specificity", "value": 0.907873, "epoch": 388}
{"metric": "train_accuracy", "value": 0.971367, "epoch": 388}
{"metric": "val_accuracy", "value": 0.886819, "epoch": 388}
{"metric": "train_mcc", "value": 0.893379, "epoch": 388}
{"metric": "val_mcc", "value": 0.602128, "epoch": 388}
{"metric": "train_auc", "value": 0.996105, "epoch": 388}
{"metric": "val_auc", "value": 0.928504, "epoch": 388}
FOLD 5 -- EPOCH 388 -- TRAINING
FOLD 5 EPOCH 388 -- VALIDATING
{"metric": "train_loss", "value": 0.082347, "epoch": 389}
{"metric": "val_loss", "value": 0.211946, "epoch": 389}
{"metric": "train_sensitivity", "value": 0.965432, "epoch": 389}
{"metric": 

FOLD 5 EPOCH 398 -- VALIDATING
{"metric": "train_loss", "value": 0.084084, "epoch": 399}
{"metric": "val_loss", "value": 0.211986, "epoch": 399}
{"metric": "train_sensitivity", "value": 0.953086, "epoch": 399}
{"metric": "val_sensitivity", "value": 0.752475, "epoch": 399}
{"metric": "train_specificity", "value": 0.970699, "epoch": 399}
{"metric": "val_specificity", "value": 0.906198, "epoch": 399}
{"metric": "train_accuracy", "value": 0.968146, "epoch": 399}
{"metric": "val_accuracy", "value": 0.883954, "epoch": 399}
{"metric": "train_mcc", "value": 0.880039, "epoch": 399}
{"metric": "val_mcc", "value": 0.591728, "epoch": 399}
{"metric": "train_auc", "value": 0.995141, "epoch": 399}
{"metric": "val_auc", "value": 0.927525, "epoch": 399}
FOLD 5 -- EPOCH 399 -- TRAINING
FOLD 5 EPOCH 399 -- VALIDATING
{"metric": "train_loss", "value": 0.078911, "epoch": 400}
{"metric": "val_loss", "value": 0.211425, "epoch": 400}
{"metric": "train_sensitivity", "value": 0.982716, "epoch": 400}
{"metric": 

FOLD 5 EPOCH 409 -- VALIDATING
{"metric": "train_loss", "value": 0.080136, "epoch": 410}
{"metric": "val_loss", "value": 0.210675, "epoch": 410}
{"metric": "train_sensitivity", "value": 0.967901, "epoch": 410}
{"metric": "val_sensitivity", "value": 0.762376, "epoch": 410}
{"metric": "train_specificity", "value": 0.973211, "epoch": 410}
{"metric": "val_specificity", "value": 0.907873, "epoch": 410}
{"metric": "train_accuracy", "value": 0.972441, "epoch": 410}
{"metric": "val_accuracy", "value": 0.886819, "epoch": 410}
{"metric": "train_mcc", "value": 0.896545, "epoch": 410}
{"metric": "val_mcc", "value": 0.602128, "epoch": 410}
{"metric": "train_auc", "value": 0.995987, "epoch": 410}
{"metric": "val_auc", "value": 0.927509, "epoch": 410}
FOLD 5 -- EPOCH 410 -- TRAINING
FOLD 5 EPOCH 410 -- VALIDATING
{"metric": "train_loss", "value": 0.077305, "epoch": 411}
{"metric": "val_loss", "value": 0.211596, "epoch": 411}
{"metric": "train_sensitivity", "value": 0.977778, "epoch": 411}
{"metric": 

from IPython.display import display
!['caption'](https://pbs.twimg.com/media/En8O1KbXMAAvOOv.jpg)

In [10]:
#        sensitivity  specificity  accuracy       mcc       auc
# train     0.959985     0.965338  0.964561  0.870052  0.993749
# val       0.735236     0.894847  0.871716  0.560142  0.915116
# load      0.756902     0.891834  0.872287  0.570648  0.916742

SyntaxError: invalid syntax (<ipython-input-10-587159ce2e17>, line 1)

***SINGLE RUN CHECK***

In [6]:
from single_run import *

train_data, val_data = train_validation_split(args.dataset)
train_dataset = EGFRDataset(train_data)
val_dataset = EGFRDataset(val_data)
if torch.cuda.is_available():
    train_device = 'cuda'
    val_device = 'cuda'
else:
    train_device = 'cpu'
    val_device = 'cpu'

if args.mode == 'train':
    train_validate_united(train_dataset,
                          val_dataset,
                          train_device,
                          val_device,
                          args.use_mat,
                          args.use_mord,
                          args.opt,
                          int(args.epochs),
                          int(args.batchsize),
                          {'sensitivity': sensitivity, 'specificity': specificity,
                           'accuracy': accuracy, 'mcc': mcc, 'auc': auc},
                          args.hashcode,
                          args.lr)
else:
    pred_dataset = EGFRDataset(args.dataset)
    y_pred = predict(pred_dataset, args.model_path, train_device)
    y_true = pred_dataset.label

    plot_roc_curve(y_true, y_pred, args.model_path.split('/')[-1])
    plot_precision_recall(y_true, y_pred, args.model_path.split('/')[-1])

0 -- TRAINING ==============>
EPOCH 0 -- VALIDATION ==============>
{"metric": "train_loss", "value": 0.648163, "epoch": 1}
{"metric": "val_loss", "value": 0.606566, "epoch": 1}
{"metric": "train_sensitivity", "value": 0.002451, "epoch": 1}
{"metric": "val_sensitivity", "value": 0.000000, "epoch": 1}
{"metric": "train_specificity", "value": 0.994549, "epoch": 1}
{"metric": "val_specificity", "value": 1.000000, "epoch": 1}
{"metric": "train_accuracy", "value": 0.849624, "epoch": 1}
{"metric": "val_accuracy", "value": 0.859800, "epoch": 1}
{"metric": "train_mcc", "value": -0.015002, "epoch": 1}
{"metric": "val_mcc", "value": 0.000000, "epoch": 1}
{"metric": "train_auc", "value": 0.471786, "epoch": 1}
{"metric": "val_auc", "value": 0.481307, "epoch": 1}
Save done!
1 -- TRAINING ==============>
EPOCH 1 -- VALIDATION ==============>
{"metric": "train_loss", "value": 0.501539, "epoch": 2}
{"metric": "val_loss", "value": 0.469787, "epoch": 2}
{"metric": "train_sensitivity", "value": 0.000000,

EPOCH 11 -- VALIDATION ==============>
{"metric": "train_loss", "value": 0.393729, "epoch": 12}
{"metric": "val_loss", "value": 0.374693, "epoch": 12}
{"metric": "train_sensitivity", "value": 0.000000, "epoch": 12}
{"metric": "val_sensitivity", "value": 0.000000, "epoch": 12}
{"metric": "train_specificity", "value": 1.000000, "epoch": 12}
{"metric": "val_specificity", "value": 1.000000, "epoch": 12}
{"metric": "train_accuracy", "value": 0.853921, "epoch": 12}
{"metric": "val_accuracy", "value": 0.859800, "epoch": 12}
{"metric": "train_mcc", "value": 0.000000, "epoch": 12}
{"metric": "val_mcc", "value": 0.000000, "epoch": 12}
{"metric": "train_auc", "value": 0.689229, "epoch": 12}
{"metric": "val_auc", "value": 0.795069, "epoch": 12}
Save done!
12 -- TRAINING ==============>
EPOCH 12 -- VALIDATION ==============>
{"metric": "train_loss", "value": 0.389660, "epoch": 13}
{"metric": "val_loss", "value": 0.372052, "epoch": 13}
{"metric": "train_sensitivity", "value": 0.000000, "epoch": 13}


EPOCH 22 -- VALIDATION ==============>
{"metric": "train_loss", "value": 0.361688, "epoch": 23}
{"metric": "val_loss", "value": 0.348259, "epoch": 23}
{"metric": "train_sensitivity", "value": 0.000000, "epoch": 23}
{"metric": "val_sensitivity", "value": 0.000000, "epoch": 23}
{"metric": "train_specificity", "value": 1.000000, "epoch": 23}
{"metric": "val_specificity", "value": 1.000000, "epoch": 23}
{"metric": "train_accuracy", "value": 0.853921, "epoch": 23}
{"metric": "val_accuracy", "value": 0.859800, "epoch": 23}
{"metric": "train_mcc", "value": 0.000000, "epoch": 23}
{"metric": "val_mcc", "value": 0.000000, "epoch": 23}
{"metric": "train_auc", "value": 0.785859, "epoch": 23}
{"metric": "val_auc", "value": 0.845360, "epoch": 23}
Save done!
23 -- TRAINING ==============>
EPOCH 23 -- VALIDATION ==============>
{"metric": "train_loss", "value": 0.355030, "epoch": 24}
{"metric": "val_loss", "value": 0.345748, "epoch": 24}
{"metric": "train_sensitivity", "value": 0.000000, "epoch": 24}


EPOCH 33 -- VALIDATION ==============>
{"metric": "train_loss", "value": 0.328400, "epoch": 34}
{"metric": "val_loss", "value": 0.328336, "epoch": 34}
{"metric": "train_sensitivity", "value": 0.000000, "epoch": 34}
{"metric": "val_sensitivity", "value": 0.000000, "epoch": 34}
{"metric": "train_specificity", "value": 1.000000, "epoch": 34}
{"metric": "val_specificity", "value": 1.000000, "epoch": 34}
{"metric": "train_accuracy", "value": 0.853921, "epoch": 34}
{"metric": "val_accuracy", "value": 0.859800, "epoch": 34}
{"metric": "train_mcc", "value": 0.000000, "epoch": 34}
{"metric": "val_mcc", "value": 0.000000, "epoch": 34}
{"metric": "train_auc", "value": 0.854908, "epoch": 34}
{"metric": "val_auc", "value": 0.865428, "epoch": 34}
Save done!
34 -- TRAINING ==============>
EPOCH 34 -- VALIDATION ==============>
{"metric": "train_loss", "value": 0.327603, "epoch": 35}
{"metric": "val_loss", "value": 0.326024, "epoch": 35}
{"metric": "train_sensitivity", "value": 0.004902, "epoch": 35}


KeyboardInterrupt: 